In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os

In [2]:
# 生成整型的属性
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


# 生成字符串属性
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))



In [3]:
# 存储数字 + 字母，训练集(不包括每个类别的后100个)
def save_to_train_tfrecord(path, name, rows=24, cols=48):
    writer = tf.python_io.TFRecordWriter(name)
    classes = os.listdir(path)
    classes.sort()
    # print(classes)
    for i, _i in enumerate(classes):
        # length = 0
        img_list_path = path + '/' + _i
        # print(img_list_path)
        a = os.listdir(img_list_path)
        for j in os.listdir(img_list_path)[:-100]:
            img_path = img_list_path + '/' + j
            # length += len(img_path)

            img = Image.open(img_path).convert('L')
            img = np.array(img.resize((rows, cols)))
            img_raw = img.tostring()
            example = tf.train.Example(features=tf.train.Features(feature={

                'image_raw': _bytes_feature(img_raw),
                'label': _int64_feature(i)}))
            writer.write(example.SerializeToString())

    writer.close()
    # return length

In [4]:
# 存储数字 + 字母，验证集(每个类别的后100个)
def save_to_validate_tfrecord(path, name, rows=24, cols=48):
    writer = tf.python_io.TFRecordWriter(name)
    classes = os.listdir(path)
    classes.sort()
    # print(classes)
    for i, _i in enumerate(classes):
        # length = 0
        img_list_path = path + '/' + _i
        # print(img_list_path)
        for j in os.listdir(img_list_path)[-100:]:
            img_path = img_list_path + '/' + j
            # length += len(img_path)
            img = Image.open(img_path).convert('L')
            img = np.array(img.resize((rows, cols)))
            img_raw = img.tostring()
            example = tf.train.Example(features=tf.train.Features(feature={

                'image_raw': _bytes_feature(img_raw),
                'label': _int64_feature(i)}))
            writer.write(example.SerializeToString())

    writer.close()
    # return length


In [5]:
class DataSet:
    def __init__(self, path):
        self.path = path

    def generate_filenamequeue(self):
        filename_queue = tf.train.string_input_producer([self.path])
        return filename_queue

    def read_from_tfrecord(self):
        reader = tf.TFRecordReader()
        _, example = reader.read(self.generate_filenamequeue())
        features = tf.parse_single_example(
            example,
            features={
                'image_raw': tf.FixedLenFeature([], tf.string),
                'label': tf.FixedLenFeature([], tf.int64),
            })

        images = tf.decode_raw(features['image_raw'], tf.uint8)
        images = tf.reshape(images, [1152])
        labels = tf.cast(features['label'], tf.int64)
        return images/255, labels

    def get_batch(self, batch_size):
        with tf.name_scope('get_batch'):
            image, label = self.read_from_tfrecord()
            images, labels = tf.train.shuffle_batch([image, label], batch_size=batch_size, num_threads=2,
                                                    capacity=1000 + 3 * batch_size, min_after_dequeue=100)

            with tf.Session() as sess:
                sess.run(tf.global_variables_initializer())
                coord = tf.train.Coordinator()
                thread = tf.train.start_queue_runners(coord=coord)
                img = sess.run(images)
                _lab = sess.run(labels)

                coord.request_stop()
                coord.join(thread)
                l = np.zeros([batch_size, 34], dtype=int)
                for i in range(batch_size):
                    l[i][_lab[i]] = 1
        return img, l

In [6]:
save_to_train_tfrecord('./data', 'train_num_letter.tfrecords')
save_to_validate_tfrecord('./data', 'validate_num_letter.tfrecords')